## Lista 2 - Regressão Logística e métodos estatísticos
Aprendizagem de Máquina - 2023.1

José Renato S. Freitas

In [493]:
import numpy as np
from numpy import array
import pandas as pd
import math
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import accuracy_score
from scipy.stats import multivariate_normal
import warnings
warnings.filterwarnings("ignore")

In [183]:
escalar_minmax = MinMaxScaler()
K = 10

In [184]:
#funções
def sigmoide(z):
  """Função logística Sigmóide, y^i = o(wTxi), o(z) = 1/1 + exp(-z)"""
  return 1/(1 + np.exp(-z))


def custo_cross_entropia_binaria(y, y_pred):
  """Função Custo Cross Entropy, Regressão Logística Binária"""
  sujeira = 1.0e-18 
  return -np.mean(y*(np.log(sujeira + y_pred)) + (1 - y)*np.log(sujeira + (1 - y_pred)))


## Questão 1

In [185]:
dataset = np.genfromtxt("./breastcancer.csv", delimiter=',', skip_header=1)
x = dataset[:, :30]
X = escalar_minmax.fit_transform(x)
y = dataset[:, -1]

In [186]:
epocas = 1000
alfa = .3
custos = []

def regressao_logistica(X, y, W):
  n = X.shape[0]
  for i in range(epocas):
      z = X @ W
      y_pred = sigmoide(z)
      erro = y - y_pred
      
      W = W + alfa*((X.T @ erro)/n)
      custo = custo_cross_entropia_binaria(y, y_pred)
      custos.append([i, custo])
  return W

kfold = KFold(n_splits=10, random_state=42, shuffle=True)
fold = 0
classe_k = []
acuracia = 0.0
acuracia_classe1 = 0.0
acuracia_classe2 = 0.0
print(f'       Valor Médio Aurácia,      Desvio Padrão,      Acurácia classe 0,     Acurácia classe 1')
for train_index, test_index in kfold.split(X):
  fold += 1
  X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]

  _W = np.zeros(X_train.shape[1])

  # Treino
  result_W = regressao_logistica(X_train, y_train, _W)
  
  # Teste
  z_teste = X_test @ result_W
  y_predito = sigmoide(z_teste)
  

  # obter a acurrácia
  classes_preditas = [1 if i > 0.5 else 0 for i in y_predito]
  classe1 = [i if i == 0 else None for i in classes_preditas]
  classe2 = [i if i == 1 else None for i in classes_preditas]
  acuracia += np.sum(y_test == classes_preditas)*1.00/len(y_test)
  acuracia_classe1 += np.sum(y_test == classe1)*1.00/len(y_test)
  acuracia_classe2 += np.sum(y_test == classe2)*1.00/len(y_test)
  
  print(f'Fold {fold} [{acuracia/10}, {np.std(classes_preditas, dtype=np.float64)}, {acuracia_classe1/10}, {acuracia_classe2/10}]')

# df_historico_k_custo = pd.DataFrame(data=custos, columns=['t','c'])
# fig, (ax1) = plt.subplots(1, 1)
# fig.suptitle("Fold " + str(fold))
# fig.set_figwidth(10)
# fig.set_figheight(6)
# ax1.plot(df_historico_k_custo['t'], df_historico_k_custo['c'], color='red')
# ax1.plot(df_historico_k_custo['t'], df_historico_k_custo['c'])
# df_historico_k_custo = pd.DataFrame(data=[])

       Valor Médio Aurácia,      Desvio Padrão,      Acurácia classe 0,     Acurácia classe 1
Fold 1 [0.08070175438596491, 0.4868223482635652, 0.054385964912280704, 0.02631578947368421]
Fold 2 [0.16666666666666666, 0.4962152850431912, 0.10175438596491229, 0.0649122807017544]
Fold 3 [0.2631578947368421, 0.4772445792538752, 0.1631578947368421, 0.1]
Fold 4 [0.3614035087719298, 0.4906011036529671, 0.22280701754385968, 0.13859649122807016]
Fold 5 [0.4508771929824561, 0.4714045207910317, 0.28771929824561404, 0.16315789473684209]
Fold 6 [0.5385964912280701, 0.4868223482635652, 0.3456140350877193, 0.19298245614035087]
Fold 7 [0.631578947368421, 0.4772445792538752, 0.4087719298245614, 0.22280701754385962]
Fold 8 [0.7157894736842104, 0.4714045207910317, 0.4631578947368421, 0.25263157894736843]
Fold 9 [0.8068609022556391, 0.4883855118277623, 0.518515037593985, 0.2883458646616541]
Fold 10 [0.8943609022556391, 0.49196347549842545, 0.5720864661654136, 0.32227443609022555]


#### Análise do Discriminante Gaussiano

In [907]:
dataset = np.genfromtxt("./breastcancer.csv", delimiter=',', skip_header=1)
x = dataset[:, :30]
X = escalar_minmax.fit_transform(x)
y = dataset[:, -1]

In [189]:
def treina_analise_discriminante_gaussiano(x_treino, y_treino):
    n = y_treino.shape[0]
    numero_de_features = x_treino.shape[1]

    rotulo_das_classes_no_dataset = len(np.unique(y_treino))
    mu = np.zeros((rotulo_das_classes_no_dataset, numero_de_features))
    prob_classe = np.zeros(rotulo_das_classes_no_dataset)
    sigma = np.zeros((rotulo_das_classes_no_dataset, numero_de_features, numero_de_features))

    for classe in range(rotulo_das_classes_no_dataset):
        indices_da_classe_k = (y_train == classe)
        nk = float(np.sum(indices_da_classe_k))

        prob_classe[classe] = nk / n

        mu[classe] = np.mean(x_treino[indices_da_classe_k, :], axis=0)
       
        sigma[classe] = np.cov(x_treino[indices_da_classe_k, :], rowvar=0)

    return prob_classe, mu, sigma


def testa_analise_discriminante_gaussiano(x_tests, probs, mu, sigma):
    x_tests = x_tests.reshape(x_tests.shape[0], -1)
    rotulo_das_classes = mu.shape[0] 
    ws = np.zeros((x_tests.shape[0], rotulo_das_classes))  
    
    for rotulo in range(rotulo_das_classes): 
        normal_distribution_prob = multivariate_normal(mean=mu[rotulo], cov=sigma[rotulo])
        for i, x_test in enumerate(x_tests):
            ws[i, rotulo] = np.log(probs[rotulo]) + normal_distribution_prob.logpdf(x_test)
    predictions = np.argmax(ws, axis=1)
    return predictions

In [190]:
kfold = KFold(n_splits=10, random_state=42, shuffle=True)
fold = 0
classe_k = []
acuracia_adg = 0.0
acuracia_classe1_adg = 0.0
acuracia_classe2_adg = 0.0

print(f'       Valor Médio Aurácia,      Desvio Padrão,      Acurácia classe 0,     Acurácia classe 1')
for train_index, test_index in kfold.split(X):
  fold += 1
  X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]

  _W = np.zeros(X_train.shape[1])

  prob_classe, mu, sigma = treina_analise_discriminante_gaussiano(X_train, y_train)

  y_predito = testa_analise_discriminante_gaussiano(X_test, prob_classe, mu, sigma)
  classe1 = [i if i == 0 else None for i in y_predito]
  classe2 = [i if i == 1 else None for i in y_predito]

  acuracia_adg += np.sum(y_test == y_predito)*1.00/len(y_test)
  acuracia_classe1_adg += np.sum(y_test == classe1)*1.00/len(y_test)
  acuracia_classe2_adg += np.sum(y_test == classe2)*1.00/len(y_test)
  
  print(f'Fold {fold} [{acuracia_adg/K}, {np.std(y_predito, dtype=np.float64)}, {acuracia_classe1_adg/10}, {acuracia_classe2_adg/10}]')


       Valor Médio Aurácia,      Desvio Padrão,      Acurácia classe 0,     Acurácia classe 1
Fold 1 [0.09122807017543859, 0.4772445792538752, 0.06140350877192983, 0.02982456140350877]
Fold 2 [0.1894736842105263, 0.4980726163711716, 0.11403508771929824, 0.07543859649122805]
Fold 3 [0.28771929824561404, 0.4906011036529671, 0.1736842105263158, 0.11403508771929824]
Fold 4 [0.38596491228070173, 0.4906011036529671, 0.23333333333333334, 0.1526315789473684]
Fold 5 [0.4842105263157895, 0.4493420517496736, 0.30526315789473685, 0.1789473684210526]
Fold 6 [0.5789473684210527, 0.464829519280413, 0.37017543859649127, 0.2087719298245614]
Fold 7 [0.6771929824561405, 0.4714045207910317, 0.436842105263158, 0.24035087719298245]
Fold 8 [0.7666666666666667, 0.4868223482635652, 0.4912280701754387, 0.2754385964912281]
Fold 9 [0.8613095238095239, 0.49487165930539345, 0.5465852130325816, 0.31472431077694235]
Fold 10 [0.955952380952381, 0.4841229182759271, 0.605513784461153, 0.3504385964912281]


### Naive Bayes Gaussiano

In [195]:
dataset = np.genfromtxt("./breastcancer.csv", delimiter=',', skip_header=1)
x = dataset[:, :30]
X = escalar_minmax.fit_transform(x)
y = dataset[:, -1]

In [894]:
def treina_naive_bayes_gaussiano2(x_treino, y_treino):
    
    numero_de_features = x_treino.shape[1]
    qtde_rotulos_classes = len(np.unique(y_treino)) # qtd de rótulos distintos
    media_dk = np.zeros((qtde_rotulos_classes, numero_de_features))
    var_dk = np.zeros((qtde_rotulos_classes, numero_de_features))

    for classe_k in range(qtde_rotulos_classes):
        indices_da_classe_k = (y_train == classe_k)
        nk = sum(indices_da_classe_k)

        # Para cada dimensão d em X
        for d in range(numero_de_features):
            # obter a média de cada dimensão (feature) por classe
            media_dk[classe_k][d] = np.mean(x_treino[indices_da_classe_k, d])
            # obter a variancia de cada dimensão por classe
            var_dk[classe_k][d] = sum((x_treino[indices_da_classe_k, d] - media_dk[classe_k][d]) * (x_treino[indices_da_classe_k, d] - media_dk[classe_k][d])) / (nk - 1)
    return media_dk, var_dk

# def teste_naive_bayes_gaussiano_(x_test, y_test, media, var):
#     numero_de_features = x_test.shape[1]
#     qtde_rotulos_classes = len(np.unique(y_test)) 

#     y_predito = np.zeros(y_test.shape[0], 1)
    
#     for classe_k in range(qtde_rotulos_classes):
#         parte1 = 0.0
#         parte2 = 0.0
#         for d in range(numero_de_features):
#             # print(f'var_dk[{classe_k},{d}]: {var[classe_k][d]}')
#             log1 = np.log( 2 * np.pi * var[classe_k][d])
#             # print(f'log1: {log1}')
#             parte1 += log1
        
#             fg2 = math.pow(np.sum(x_test[:, d] - media[classe_k][d]), 2)
#             # print(f'fg2: {np.sum(fg2)}')

#             parte2 += (fg2)/var[classe_k][d]
#             # for xi in x_test[:, d]:
#             #     xi_menos_media = xi - media[classe_k][d]
#             #     parte2 += ((xi_menos_media)*(xi_menos_media)) / (var[classe_k][d])
#         print(f'probab: {np.log(1/qtde_rotulos_classes) - (parte1/2) - (parte2/2)}')
#         y_predito = np.log(0.5) - (parte1/2) - (parte2/2)
#     return y_predito


def probabilidade_x_dado_classe(x_test, media, var):
    part_1_da_equacao = 1/(np.sqrt(2 * np.pi * var))
    numerator = math.pow(x_test - media, 2)
    probabilidade = part_1_da_equacao * np.exp(-(numerator/(2*var)))
    return probabilidade

def teste_naive_bayes_gaussiano(x_test, y_test, media, var):
    numero_de_features = x_test.shape[1]
    qtde_rotulos_classes = len(np.unique(y_test)) 
    
    #priori binária
    prob_c0, prob_c1 = 1/qtde_rotulos_classes, 1/qtde_rotulos_classes

    # encontrar a evidencia
    # prob_c0 * prob_x0|c0 * prob_xn | c0 +
    # prob_c1 * prob_x0|c1 * prob_xn | c1  
    probs = dict()
    y_predito = []
    for i in range(x_test.shape[0]): # para essa entrada, qual a classe predita
        # calcular a probabilidade de xi dado a Classe k -> p(x | Ck)
        for classe_k in range(qtde_rotulos_classes):
            prob_x_ck = []
            for d in range(numero_de_features):
                prob_x_ck.append(probabilidade_x_dado_classe(x_test[i, d], media[classe_k][d], var[classe_k][d]))
                probs[classe_k] = prob_x_ck

        verossi = []
        for i in probs:
            verossi.append(np.prod(probs[i]) * prob_c0)
        maior_prob = max(verossi)
        y_predito.append(verossi.index(maior_prob))
    return y_predito

        

In [915]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
kfold = KFold(n_splits=K, random_state=42, shuffle=True)
fold = 0
classe = []
acuracia_nbg = 0.0
acuracia_classe1_nbg = 0.0
acuracia_classe2_nbg = 0.0

print(f'\tValor Médio Aurácia,      Desvio Padrão,      Acurácia classe 0,     Acurácia classe 1')
for train_index, test_index in kfold.split(X):
  fold += 1
  X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]

  # modelo_nbg = clf.fit(X_train, y_train)
  # predito = modelo_nbg.predict(X_test[:10, :30])
  # print(f'\tPredição GaussianNG do Sklearn: {predito}')

  media, var = treina_naive_bayes_gaussiano2(X_train, y_train)
  
  y_predito = teste_naive_bayes_gaussiano(X_test, y_test, media, var)

  classe0 = [i if i == 0 else None for i in y_predito]
  classe1 = [i if i == 1 else None for i in y_predito]
  # print(f'\ty_predito:                      {(y_predito[:10])}')

  acuracia_nbg += np.sum(y_test == y_predito)*1.00/len(y_test)
  acuracia_classe1_nbg += np.sum(y_test == classe1)*1.00/len(y_test)
  acuracia_classe2_nbg += np.sum(y_test == classe2)*1.00/len(y_test)
  
  print(f'Fold {fold} [{acuracia_nbg/K}, {np.std(y_predito, dtype=np.float64)}, {acuracia_classe1_nbg/K}, {acuracia_classe2_nbg/K}]')
  


	Valor Médio Aurácia,      Desvio Padrão,      Acurácia classe 0,     Acurácia classe 1
Fold 1 [0.08947368421052632, 0.4574878880843964, 0.02631578947368421, 0.0]
Fold 2 [0.18596491228070175, 0.4962152850431912, 0.07017543859649122, 0.0]
Fold 3 [0.28421052631578947, 0.4906011036529671, 0.10877192982456138, 0.0]
Fold 4 [0.38421052631578945, 0.4868223482635652, 0.14736842105263154, 0.0]
Fold 5 [0.4807017543859649, 0.4574878880843964, 0.17368421052631575, 0.0]
Fold 6 [0.5684210526315789, 0.464829519280413, 0.19999999999999996, 0.0]
Fold 7 [0.6578947368421052, 0.4868223482635652, 0.22982456140350874, 0.0]
Fold 8 [0.7491228070175439, 0.4714045207910317, 0.2631578947368421, 0.0]
Fold 9 [0.8419799498746867, 0.4841229182759271, 0.2988721804511278, 0.023214285714285715]
Fold 10 [0.9348370927318296, 0.48838551182776224, 0.33458646616541354, 0.05892857142857143]


# Questão 2

In [191]:
dataset = np.genfromtxt("./vehicle.csv", delimiter=',', skip_header=1)

In [192]:
X = dataset[:, :19]
X = escalar_minmax.fit_transform(X)
y = dataset[:, -1]

In [193]:
K = 10
alfa2 = 0.1
epocas2 = 1000
custos2 = []
from sklearn.utils.extmath import softmax

def my_softmax2(z):
  exp = np.exp(z)
  return (exp / np.sum(exp))

def custo_multi_cross_entropia(y, y_predi):
    """cross_entropy_loss"""
    return -np.mean(np.sum(y * np.log(y_predi)))

def acuracia(y, y_hat):
    return np.sum(y == y_hat)/len(y)

one_hot_encoder = OneHotEncoder(sparse=False)

def treina(X, y):
  _x = np.c_[np.ones(X.shape[0]), X]
  y_encode = one_hot_encoder.fit_transform(y.reshape(-1,1))
  W = np.zeros((_x.shape[1], y_encode.shape[1]))
  
  y_predi = None
  n = X.shape[0]
  for i in range(epocas2):
    z = _x @ W
    y_predi = softmax(z)
    erro = y_encode - y_predi
    
    W = W + alfa2*((_x.T @ erro)/n)
    custo2 = custo_multi_cross_entropia(y_encode, y_predi)
    custos2.append([i, custo2])
  return W
   

def testa(X, W):
  features = np.c_[np.ones(X.shape[0]), X]
  z = features @ W
  y_predi_ = softmax(z)
  return y_predi_

In [194]:

kfold = KFold(n_splits=K, random_state=42, shuffle=True)
acuracia_ = 0.0
acuracia_classe_zero,  acuracia_classe_um, acuracia_classe_dois, acuracia_classe_tres= 0.0, 0.0, 0.0, 0.0
print(f'       Valor Médio Aurácia,      Desvio Padrão,      Acurácia classe 0,     Acurácia classe 1, Acurácia classe 2, Acurácia classe 3')
for train_index, test_index in kfold.split(X):

  X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]
  
  ws = treina(X_train, y_train)
  
  y_ch = testa(X_test, ws)
  classes_preditas = np.array([np.argmax(y)*1.0 for y in y_ch])

  # obter a acurrácia
  classe_zero= [i if i == 0 else None for i in classes_preditas]
  classe_um = [i if i == 1 else None for i in classes_preditas]
  classe_dois = [i if i == 2 else None for i in classes_preditas]
  classe_tres = [i if i == 3 else None for i in classes_preditas]
  acuracia_ += np.sum(y_test == classes_preditas)*1.00/len(y_test)
  acuracia_classe_zero += np.sum(y_test == classe_zero)*1.00/len(y_test)
  acuracia_classe_um += np.sum(y_test == classe_um)*1.00/len(y_test)
  acuracia_classe_dois += np.sum(y_test == classe_dois)*1.00/len(y_test)
  acuracia_classe_tres += np.sum(y_test == classe_tres)*1.00/len(y_test)

  print(f'Fold {fold} [{acuracia_/K}, {np.std(classes_preditas, dtype=np.float64)}, {acuracia_classe_zero/K}, {acuracia_classe_um/K}, {acuracia_classe_dois/K}, {acuracia_classe_tres/K}]')

# df_historico_multi_custo = pd.DataFrame(data=custos2, columns=['t','c'])
# fig, (ax1) = plt.subplots(1, 1)
# fig.suptitle("Custo Cross Entroty")
# fig.set_figwidth(12)
# ax1.plot(df_historico_multi_custo['t'], df_historico_multi_custo['c'])

       Valor Médio Aurácia,      Desvio Padrão,      Acurácia classe 0,     Acurácia classe 1, Acurácia classe 2, Acurácia classe 3
Fold 10 [0.08823529411764705, 1.1642898841165181, 0.024705882352941178, 0.02235294117647059, 0.02235294117647059, 0.018823529411764704]
Fold 10 [0.1764705882352941, 1.2621725653709506, 0.049411764705882356, 0.03764705882352941, 0.03882352941176471, 0.05058823529411764]
Fold 10 [0.2752941176470588, 1.048313854585669, 0.07529411764705882, 0.06823529411764706, 0.06470588235294118, 0.06705882352941175]
Fold 10 [0.36705882352941177, 1.1942190626305431, 0.09529411764705882, 0.09058823529411765, 0.0811764705882353, 0.1]
Fold 10 [0.46352941176470586, 1.1697454843981643, 0.13058823529411764, 0.11058823529411765, 0.10235294117647058, 0.12]
Fold 10 [0.5551960784313725, 1.239182215155736, 0.16511204481792716, 0.13082633053221288, 0.11544817927170867, 0.1438095238095238]
Fold 10 [0.6468627450980392, 1.1639911011028874, 0.18773109243697478, 0.14630252100840338, 0.144019